In [16]:
import polars as pl 
import pandas as pd 
from data import NB_NT

data = pl.read_csv("output/output.csv", infer_schema_length = 10000)



In [2]:
import polars as pl

In [ ]:
def get_upstream(data_path = "output/output.csv", nb_codons : int = 3):

    data = pl.read_csv(data_path, infer_schema_length = 10000).filter(pl.col("ribostartLocalisation") == "exon")

    threshold_f1 = ( nb_codons * 3 ) + 1
    threshold_f2 = ( nb_codons * 3 ) + 2



    sorted = data.filter(
        
        ((pl.col('upstream').apply(lambda x: len(x)) >= threshold_f1) & (pl.col('rel_frame') == 1)) |
        ((pl.col('upstream').apply(lambda x: len(x)) >= threshold_f2) & (pl.col('rel_frame') == 2))

    )

    sorted = sorted.with_columns((pl.col('upstream') + pl.col('start_seq')).alias('sequence'))

    codons = []


    # Case F1 validated
    for orf in sorted.filter(pl.col('rel_frame') == 1).iter_rows(named = True):
        codons = []
        for i in range(nb_codons):

            ribostart = len(orf['sequence']) - 3
            start = ribostart - 1 - 3 * i
            end = ribostart + 2 - 3 * i
            codons.append(orf['sequence'][start:end])
    
        

    # Case F2 validated
    for orf in sorted.filter(pl.col('rel_frame') == 2).iter_rows(named = True):
        for i in range(nb_codons):

            ribostart = len(orf['sequence']) - 3
            start = ribostart - 2 - 3 * i
            end = ribostart + 1 - 3 * i
            codons.append(orf['sequence'][start:end])



    return codons

        
def get_downstream(data_path = "output/output.csv", nb_codons : int = 3):

    data = pl.read_csv(data_path, infer_schema_length = 10000).filter(pl.col("ribostartLocalisation") == "exon")

    threshold_f1 = ( nb_codons * 3 )  + 2
    threshold_f2 = ( nb_codons * 3 ) + 1
    

    sorted = data.filter(
        
        ((pl.col('downstream').apply(lambda x: len(x)) >= threshold_f1) & (pl.col('rel_frame') == 1)) |
        ((pl.col('downstream').apply(lambda x: len(x)) >= threshold_f2) & (pl.col('rel_frame') == 2))

    )

    sorted = sorted.with_columns((pl.col('start_seq') + pl.col('downstream')).alias('sequence'))


    # Case F1 validated
    for orf in sorted.filter(pl.col('rel_frame') == 1).iter_rows(named = True):

        codons = []
        for i in range(nb_codons):

            ribostart = 0
            start = ribostart + 2 + 3 * i
            end = ribostart + 5 + 3 * i
            codons.append(orf['sequence'][start:end])
        print(f"{orf['ID']}: {codons}")


    # Case F2 validated
    for orf in sorted.filter(pl.col('rel_frame') == 2).iter_rows(named = True):

        codons = []

        for i in range(nb_codons):

            ribostart = 0
            start = ribostart + 1 + 3 * i
            end = ribostart + 4 + 3 * i
            codons.append(orf['sequence'][start:end])



    return None       

get_downstream()      


In [47]:
from data import ORFS, RIBO, GFF_POLARS, ORF_DF_COLUMNS
from utils import check_double_overlap

ORFS = (
        ORFS
        .filter(pl.col("Type") == "nc_ovp_same-CDS")
        .with_columns([
            pl.col("ID").apply(lambda value: value.split("_")[3]).alias("Phase"), # Extract absolute strand phase from the ID generated by ORFMine
            pl.lit("NA").alias("Ovp_gene") # Create a column for the gene ID that will be filled by check_double_overlap()
        ])
    )

ORFS = ORFS.select(ORF_DF_COLUMNS).apply(check_double_overlap) # Use .apply() method to leverage parallelization
ORFS.columns = ORF_DF_COLUMNS


joined = ORFS.join(RIBO, left_on = "ID", right_on = "Seq_ID", how = "inner")

test = joined.filter(
    pl.col("MSMS") == 1
)





In [49]:
for overlapped_feature_name , data in test: # This loop returns the name by which data is groupped, and the data itself as a polars dataframe

        # If overlapped feature is not a gene ( = transposable_element for example ) it's not stored
        if "gene" not in GFF_POLARS.filter(pl.col("ID") == overlapped_feature_name)["Type"].unique().to_list(): 

            print(data["ID"].to_list())
            print(overlapped_feature_name)

['chrXI_-_135687-135866_1_nc_ovp_same-CDS']
Two_or_more_genes
